In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
from torchvision.models import resnet50
from tqdm import tqdm

def get_cifar100_data_loaders(data_dir='./data'):
    train_tf = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])

    train_ds = torchvision.datasets.CIFAR100(data_dir, train=True, download=True, transform=train_tf)
    test_ds = torchvision.datasets.CIFAR100(data_dir, train=False, download=True, transform=test_tf)

    train_size = len(train_ds)
    indices = list(range(train_size))
    np.random.seed(42)
    np.random.shuffle(indices)
    split = int(0.1 * train_size)
    train_idx, val_idx = indices[split:], indices[:split]

    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=False)
    train_loader = DataLoader(train_ds, batch_size=128, sampler=SubsetRandomSampler(train_idx), num_workers=4, pin_memory=True)
    val_loader = DataLoader(train_ds, batch_size=256, sampler=SubsetRandomSampler(val_idx), num_workers=4, pin_memory=True)

    return train_loader, val_loader, test_loader, train_idx

train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

Files already downloaded and verified
Files already downloaded and verified


# FP32 BASELINE

In [6]:


def evaluate_accuracy_and_latency(model, loader, device, num_batches=None, desc="Evaluating"):
    model.eval()
    model.to(device)
    correct = total = 0
    latencies = []
    with torch.no_grad():
        for i, (imgs, labels) in enumerate(tqdm(loader, desc=desc, leave=False)):
            if num_batches is not None and i >= num_batches:
                break
            imgs, labels = imgs.to(device), labels.to(device)
            start_time = time.time()
            outputs = model(imgs)
            latency = (time.time() - start_time) * 1000  # Latency in ms/batch
            latencies.append(latency)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100.0 * correct / total
    avg_latency = sum(latencies) / len(latencies)
    return accuracy, avg_latency

def main():
    train_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    assert train_device.type == 'cuda', "GPU not available for training/validation"
    test_device = torch.device('cpu')

    model = resnet50(pretrained=False, num_classes=100)
    # Modify conv1 and maxpool for 32x32 inputs
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    model.maxpool = nn.Identity()  # Remove maxpool to preserve spatial dimensions
    model.to(train_device)

    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    best_val_acc = 0.0
    best_model_path = 'best_fp32_resnet50_cifar100_32x32_200.pth'
    epochs = 200
    patience = 20
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for imgs, targets in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=True):
            imgs, targets = imgs.to(train_device), targets.to(train_device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        val_acc, _ = evaluate_accuracy_and_latency(model, val_loader, train_device, desc="Validating")
        print(f"Epoch {epoch+1}/{epochs} – Loss: {running_loss/len(train_idx):.4f}, Val Accuracy: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best FP32 model with Val Accuracy: {best_val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"No improvement in validation accuracy. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        scheduler.step()

    model.load_state_dict(torch.load(best_model_path))
    test_acc, avg_latency = evaluate_accuracy_and_latency(model, test_loader, test_device, desc="Testing FP32")
    print(f"\nBest FP32 Model – Test Accuracy: {test_acc:.2f}%")
    print(f"Average FP32 Inference Latency (CPU): {avg_latency:.2f} ms/batch")

if __name__ == '__main__':
    main()

Training Epoch 1/200: 100%|██████████| 352/352 [00:19<00:00, 17.78it/s]


Epoch 1/200 – Loss: 4.1701, Val Accuracy: 13.94%
Saved best FP32 model with Val Accuracy: 13.94%


Training Epoch 2/200: 100%|██████████| 352/352 [00:19<00:00, 17.78it/s]


Epoch 2/200 – Loss: 3.5226, Val Accuracy: 24.60%
Saved best FP32 model with Val Accuracy: 24.60%


Training Epoch 3/200: 100%|██████████| 352/352 [00:20<00:00, 17.56it/s]


...

Training Epoch 98/200: 100%|██████████| 352/352 [00:19<00:00, 17.79it/s]


Epoch 98/200 – Loss: 0.7989, Val Accuracy: 71.82%
No improvement in validation accuracy. Patience counter: 19/20


Training Epoch 99/200: 100%|██████████| 352/352 [00:19<00:00, 17.62it/s]


Epoch 99/200 – Loss: 0.8020, Val Accuracy: 72.46%
No improvement in validation accuracy. Patience counter: 20/20
Early stopping triggered after 99 epochs.



Best FP32 Model – Test Accuracy: 72.05%
Average FP32 Inference Latency (CPU): 261.23 ms/batch


# QAT 

In [7]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.quantization import (
    prepare_qat,
    convert,
    enable_observer,
    disable_observer,
    enable_fake_quant,
    disable_fake_quant,
)
from torchvision.models.quantization import resnet50
import numpy as np
from tqdm import tqdm

torch.backends.quantized.engine = 'fbgemm'

def get_cifar100_data_loaders(data_dir='./data'):
    train_tf = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])

    train_ds = torchvision.datasets.CIFAR100(data_dir, train=True, download=True, transform=train_tf)
    test_ds = torchvision.datasets.CIFAR100(data_dir, train=False, download=True, transform=test_tf)

    train_size = len(train_ds)
    indices = list(range(train_size))
    np.random.seed(42)
    np.random.shuffle(indices)
    split = int(0.1 * train_size)
    train_idx, val_idx = indices[split:], indices[:split]

    train_loader = DataLoader(train_ds, batch_size=128, sampler=SubsetRandomSampler(train_idx), num_workers=4, pin_memory=True)
    val_loader = DataLoader(train_ds, batch_size=256, sampler=SubsetRandomSampler(val_idx), num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=False)

    return train_loader, val_loader, test_loader, train_idx

# train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

def evaluate_accuracy_and_latency(model, loader, device, num_batches=None, desc="Evaluating"):
    model.eval()
    model.to(device)
    correct = total = 0
    latencies = []
    with torch.no_grad():
        for i, (imgs, labels) in enumerate(tqdm(loader, desc=desc, leave=True)):
            if num_batches is not None and i >= num_batches:
                break
            imgs, labels = imgs.to(device), labels.to(device)
            start_time = time.time()
            outputs = model(imgs)
            latency = (time.time() - start_time) * 1000  # Latency in ms/batch
            latencies.append(latency)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100.0 * correct / total
    avg_latency = sum(latencies) / len(latencies)
    return accuracy, avg_latency

def main():
    train_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    assert train_device.type == 'cuda', "GPU not available for training/validation"
    test_device = torch.device('cpu')

    model = resnet50(pretrained=False, quantize=False, num_classes=100)
    # Modify conv1 and maxpool for 32x32 inputs
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    model.maxpool = nn.Identity()  # Remove maxpool to preserve spatial dimensions
    model.fuse_model()
    model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    prepare_qat(model, inplace=True)
    model.to(train_device)

    model.eval()
    disable_fake_quant(model)
    enable_observer(model)
    with torch.no_grad():
        for i, (imgs, _) in enumerate(tqdm(train_loader, desc="Calibrating", leave=False)):
            if i >= 200:
                break
            imgs = imgs.to(train_device)
            model(imgs)
    disable_observer(model)
    enable_fake_quant(model)

    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    best_val_acc = 0.0
    best_model_path = 'qat_resnet50_cifar100_float_32x32_200.pth'
    epochs = 200
    patience = 20
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        enable_fake_quant(model)
        running_loss = 0.0
        for imgs, targets in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=False):
            imgs, targets = imgs.to(train_device), targets.to(train_device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        val_acc, _ = evaluate_accuracy_and_latency(model, val_loader, train_device, desc="Validating")
        print(f"Epoch {epoch+1}/{epochs} – Loss: {running_loss/len(train_idx):.4f}, ValAcc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best QAT model with ValAcc: {best_val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"No improvement in validation accuracy. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        scheduler.step()

    model.load_state_dict(torch.load(best_model_path))
    fake_quant_acc, _ = evaluate_accuracy_and_latency(model, test_loader, train_device, desc="Testing Fake-Quantized")
    print(f"\nBest Fake-Quantized Model – Test Accuracy: {fake_quant_acc:.2f}%")

    model.cpu()
    model.eval()
    quantized_model = convert(model, inplace=False)

    int8_acc, int8_latency = evaluate_accuracy_and_latency(quantized_model, test_loader, test_device, desc="Testing INT8")
    print(f"Best INT8 Model – Test Accuracy: {int8_acc:.2f}%")
    print(f"Average INT8 Inference Latency (CPU): {int8_latency:.2f} ms/batch")

    # torch.save(quantized_model.state_dict(), 'qat_resnet50_cifar100_int8_final_32x32.pth')
    # TODO ONNX integration
if __name__ == '__main__':
    main()

Validating: 100%|██████████| 20/20 [00:01<00:00, 10.98it/s]            


Epoch 1/200 – Loss: 4.1797, ValAcc: 14.60%
Saved best QAT model with ValAcc: 14.60%


Validating: 100%|██████████| 20/20 [00:01<00:00, 11.55it/s]            


Epoch 2/200 – Loss: 3.5307, ValAcc: 21.30%
Saved best QAT model with ValAcc: 21.30%


Validating: 100%|██████████| 20/20 [00:01<00:00, 11.09it/s]            


...

Epoch 114/200 – Loss: 0.8223, ValAcc: 70.20%
No improvement in validation accuracy. Patience counter: 20/20
Early stopping triggered after 114 epochs.


Testing Fake-Quantized: 100%|██████████| 1250/1250 [00:33<00:00, 36.88it/s]



Best Fake-Quantized Model – Test Accuracy: 73.78%


Testing INT8: 100%|██████████| 1250/1250 [02:45<00:00,  7.54it/s]


Best INT8 Model – Test Accuracy: 73.67%
Average INT8 Inference Latency (CPU): 130.13 ms/batch


# QAT +KD

In [10]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.quantization import (
    prepare_qat,
    convert,
    enable_observer,
    disable_observer,
    enable_fake_quant,
    disable_fake_quant,
)
from torchvision.models.quantization import resnet50
import numpy as np
from tqdm import tqdm

torch.backends.quantized.engine = 'fbgemm'

def get_cifar100_data_loaders(data_dir='./data'):
    train_tf = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])

    train_ds = torchvision.datasets.CIFAR100(data_dir, train=True, download=True, transform=train_tf)
    test_ds = torchvision.datasets.CIFAR100(data_dir, train=False, download=True, transform=test_tf)

    train_size = len(train_ds)
    indices = list(range(train_size))
    np.random.seed(42)
    np.random.shuffle(indices)
    split = int(0.1 * train_size)
    train_idx, val_idx = indices[split:], indices[:split]

    train_loader = DataLoader(train_ds, batch_size=128, sampler=SubsetRandomSampler(train_idx), num_workers=4, pin_memory=True)
    val_loader = DataLoader(train_ds, batch_size=256, sampler=SubsetRandomSampler(val_idx), num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=False)

    return train_loader, val_loader, test_loader, train_idx

def evaluate_accuracy_and_latency(model, loader, device, num_batches=None, desc="Evaluating"):
    model.eval()
    model.to(device)
    correct = total = 0
    latencies = []
    with torch.no_grad():
        for i, (imgs, labels) in enumerate(tqdm(loader, desc=desc, leave=False)):
            if num_batches is not None and i >= num_batches:
                break
            imgs, labels = imgs.to(device), labels.to(device)
            start_time = time.time()
            outputs = model(imgs)
            latency = (time.time() - start_time) * 1000  # Latency in ms/batch
            latencies.append(latency)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100.0 * correct / total
    avg_latency = sum(latencies) / len(latencies) if latencies else 0.0
    return accuracy, avg_latency

class KDLoss(nn.Module):
    def __init__(self, temperature=3.0, alpha=0.7):
        super(KDLoss, self).__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss(label_smoothing=0.1)
        self.kld_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_outputs, teacher_outputs, labels):
        ce = self.ce_loss(student_outputs, labels)
        kld = self.kld_loss(
            F.log_softmax(student_outputs / self.temperature, dim=1),
            F.softmax(teacher_outputs / self.temperature, dim=1)
        ) * (self.temperature ** 2)
        return self.alpha * ce + (1 - self.alpha) * kld

def main():
    train_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    assert train_device.type == 'cuda', "GPU not available for training/validation"
    test_device = torch.device('cpu')

    # train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

    # Load pre-trained FP32 teacher model
    teacher_model = resnet50(pretrained=False, num_classes=100)
    # Modify teacher model for 32x32 inputs
    teacher_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    teacher_model.maxpool = nn.Identity()
    teacher_model.load_state_dict(torch.load('best_fp32_resnet50_cifar100_32x32_200.pth'))
    teacher_model.to(train_device)
    teacher_model.eval()

    # Initialize QAT student model
    student_model = resnet50(pretrained=False, quantize=False, num_classes=100)
    # Modify student model for 32x32 inputs
    student_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    student_model.maxpool = nn.Identity()
    student_model.fuse_model()
    student_model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    prepare_qat(student_model, inplace=True)
    student_model.to(train_device)

    # Calibration
    student_model.eval()
    disable_fake_quant(student_model)
    enable_observer(student_model)
    with torch.no_grad():
        for i, (imgs, _) in enumerate(tqdm(train_loader, desc="Calibrating", leave=True)):
            if i >= 200:
                break
            imgs = imgs.to(train_device)
            student_model(imgs)
    disable_observer(student_model)
    enable_fake_quant(student_model)

    optimizer = optim.AdamW(student_model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    criterion = KDLoss(temperature=3.0, alpha=0.7)
    best_val_acc = 0.0
    best_model_path = 'qat_kd_resnet50_cifar100_float_32x32_200.pth'
    epochs = 200
    patience = 20
    patience_counter = 0

    for epoch in range(epochs):
        student_model.train()
        enable_fake_quant(student_model)
        running_loss = 0.0
        for imgs, targets in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=True):
            imgs, targets = imgs.to(train_device), targets.to(train_device)
            optimizer.zero_grad()
            with torch.no_grad():
                teacher_outputs = teacher_model(imgs)
            student_outputs = student_model(imgs)
            loss = criterion(student_outputs, teacher_outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        val_acc, _ = evaluate_accuracy_and_latency(student_model, val_loader, train_device, desc="Validating")
        print(f"Epoch {epoch+1}/{epochs} – Loss: {running_loss/len(train_idx):.4f}, ValAcc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student_model.state_dict(), best_model_path)
            print(f"Saved best QAT+KD model with ValAcc: {best_val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"No improvement in validation accuracy. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        scheduler.step()

    student_model.load_state_dict(torch.load(best_model_path))
    fake_quant_acc, fake_quant_latency = evaluate_accuracy_and_latency(student_model, test_loader, train_device, desc="Testing Fake-Quantized")
    print(f"\nBest Fake-Quantized Model – Test Accuracy: {fake_quant_acc:.2f}%")
    print(f"Average Fake-Quantized Inference Latency (GPU): {fake_quant_latency:.2f} ms/batch")

    student_model.cpu()
    student_model.eval()
    quantized_model = convert(student_model, inplace=False)

    int8_acc, int8_latency = evaluate_accuracy_and_latency(quantized_model, test_loader, test_device, desc="Testing INT8")
    print(f"Best INT8 Model – Test Accuracy: {int8_acc:.2f}%")
    print(f"Average INT8 Inference Latency (CPU): {int8_latency:.2f} ms/batch")

    # torch.save(quantized_model.state_dict(), 'qat_kd_resnet50_cifar100_int8_final_32x32_200.pth')
    # TODO ONNX integration
if __name__ == '__main__':
    main()

Epoch 1/200 – Loss: 3.3097, ValAcc: 15.38%
Saved best QAT+KD model with ValAcc: 15.38%


Epoch 2/200 – Loss: 2.8383, ValAcc: 26.56%
Saved best QAT+KD model with ValAcc: 26.56%


Epoch 3/200 – Loss: 2.5279, ValAcc: 31.40%
Saved best QAT+KD model with ValAcc: 31.40%


Epoch 4/200 – Loss: 2.2946, ValAcc: 37.94%
Saved best QAT+KD model with ValAcc: 37.94%


Epoch 5/200 – Loss: 2.0871, ValAcc: 43.28%
Saved best QAT+KD model with ValAcc: 43.28%


...

Epoch 125/200 – Loss: 0.5749, ValAcc: 73.28%
No improvement in validation accuracy. Patience counter: 18/20


Epoch 126/200 – Loss: 0.5742, ValAcc: 73.14%
No improvement in validation accuracy. Patience counter: 19/20


Epoch 127/200 – Loss: 0.5740, ValAcc: 72.60%
No improvement in validation accuracy. Patience counter: 20/20
Early stopping triggered after 127 epochs.



Best Fake-Quantized Model – Test Accuracy: 73.96%
Average Fake-Quantized Inference Latency (GPU): 24.97 ms/batch


Best INT8 Model – Test Accuracy: 73.90%
Average INT8 Inference Latency (CPU): 130.38 ms/batch


# ENKD

In [2]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.quantization import (
    prepare_qat,
    convert,
    enable_observer,
    disable_observer,
    enable_fake_quant,
    disable_fake_quant,
)
from torchvision.models.quantization import resnet50
import numpy as np
from tqdm import tqdm

torch.backends.quantized.engine = 'fbgemm'

def get_cifar100_data_loaders(data_dir='./data'):
    train_tf = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])

    train_ds = torchvision.datasets.CIFAR100(data_dir, train=True, download=True, transform=train_tf)
    test_ds = torchvision.datasets.CIFAR100(data_dir, train=False, download=True, transform=test_tf)

    train_size = len(train_ds)
    indices = list(range(train_size))
    np.random.seed(42)
    np.random.shuffle(indices)
    split = int(0.1 * train_size)
    train_idx, val_idx = indices[split:], indices[:split]

    train_loader = DataLoader(train_ds, batch_size=128, sampler=SubsetRandomSampler(train_idx), num_workers=4, pin_memory=True)
    val_loader = DataLoader(train_ds, batch_size=256, sampler=SubsetRandomSampler(val_idx), num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=False)

    return train_loader, val_loader, test_loader, train_idx

def evaluate_accuracy_and_latency(model, loader, device, num_batches=None, desc="Evaluating"):
    model.eval()
    model.to(device)
    correct = total = 0
    latencies = []
    with torch.no_grad():
        for i, (imgs, labels) in enumerate(tqdm(loader, desc=desc, leave=False)):
            if num_batches is not None and i >= num_batches:
                break
            imgs, labels = imgs.to(device), labels.to(device)
            start_time = time.time()
            outputs = model(imgs)
            latency = (time.time() - start_time) * 1000  # Latency in ms/batch
            latencies.append(latency)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100.0 * correct / total
    avg_latency = sum(latencies) / len(latencies) if latencies else 0.0
    return accuracy, avg_latency

class EntKDLoss(nn.Module):
    def __init__(self, base_temperature=3.0, alpha=0.7, beta=0.1):
        super(EntKDLoss, self).__init__()
        self.base_temperature = base_temperature
        self.alpha = alpha
        self.beta = beta
        self.ce_loss = nn.CrossEntropyLoss(label_smoothing=0.1)
        self.kld_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_outputs, teacher_outputs, labels):
        # Compute teacher entropy
        teacher_probs = F.softmax(teacher_outputs, dim=1)
        entropy = -torch.sum(teacher_probs * torch.log(teacher_probs + 1e-10), dim=1).mean()
        
        # Dynamic temperature
        temperature = self.base_temperature / (1 + self.beta * entropy)
        temperature = torch.clamp(temperature, min=1.0, max=10.0)  # Prevent extreme values

        # Compute loss
        ce = self.ce_loss(student_outputs, labels)
        kld = self.kld_loss(
            F.log_softmax(student_outputs / temperature, dim=1),
            F.softmax(teacher_outputs / temperature, dim=1)
        ) * (temperature ** 2)
        return self.alpha * ce + (1 - self.alpha) * kld

def main():
    train_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    assert train_device.type == 'cuda', "GPU not available for training/validation"
    test_device = torch.device('cpu')

    train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

    # Load pre-trained FP32 teacher model
    teacher_model = resnet50(pretrained=False, num_classes=100)
    teacher_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    teacher_model.maxpool = nn.Identity()
    teacher_model.load_state_dict(torch.load('best_fp32_resnet50_cifar100_32x32_200.pth'))
    teacher_model.to(train_device)
    teacher_model.eval()

    # Initialize QAT student model
    student_model = resnet50(pretrained=False, quantize=False, num_classes=100)
    student_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    student_model.maxpool = nn.Identity()
    student_model.fuse_model()
    student_model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    prepare_qat(student_model, inplace=True)
    student_model.to(train_device)

    # Calibration
    student_model.eval()
    disable_fake_quant(student_model)
    enable_observer(student_model)
    with torch.no_grad():
        for i, (imgs, _) in enumerate(tqdm(train_loader, desc="Calibrating", leave=False)):
            if i >= 200:
                break
            imgs = imgs.to(train_device)
            student_model(imgs)
    disable_observer(student_model)
    enable_fake_quant(student_model)

    optimizer = optim.AdamW(student_model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    criterion = EntKDLoss(base_temperature=3.0, alpha=0.3, beta=0.1)  # Fixed: temperature -> base_temperature
    best_val_acc = 0.0
    best_model_path = 'qat_entkd_resnet50_cifar100_float_32x32_200.pth'
    epochs = 200
    patience = 20
    patience_counter = 0

    for epoch in range(epochs):
        student_model.train()
        enable_fake_quant(student_model)
        running_loss = 0.0
        for imgs, targets in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=False):
            imgs, targets = imgs.to(train_device), targets.to(train_device)
            optimizer.zero_grad()
            with torch.no_grad():
                teacher_outputs = teacher_model(imgs)
            student_outputs = student_model(imgs)
            loss = criterion(student_outputs, teacher_outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        val_acc, _ = evaluate_accuracy_and_latency(student_model, val_loader, train_device, desc="Validating")
        print(f"Epoch {epoch+1}/{epochs} – Loss: {running_loss/len(train_idx):.4f}, ValAcc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student_model.state_dict(), best_model_path)
            print(f"Saved best QAT+EntKD model with ValAcc: {best_val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"No improvement in validation accuracy. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        scheduler.step()

    student_model.load_state_dict(torch.load(best_model_path))
    fake_quant_acc, fake_quant_latency = evaluate_accuracy_and_latency(student_model, test_loader, train_device, desc="Testing Fake-Quantized")
    print(f"\nBest Fake-Quantized Model – Test Accuracy: {fake_quant_acc:.2f}%")
    print(f"Average Fake-Quantized Inference Latency (GPU): {fake_quant_latency:.2f} ms/batch")

    student_model.cpu()
    student_model.eval()
    quantized_model = convert(student_model, inplace=False)

    int8_acc, int8_latency = evaluate_accuracy_and_latency(quantized_model, test_loader, test_device, desc="Testing INT8")
    print(f"Best INT8 Model – Test Accuracy: {int8_acc:.2f}%")
    print(f"Average INT8 Inference Latency (CPU): {int8_latency:.2f} ms/batch")

    # torch.save(quantized_model.state_dict(), 'qat_entkd_resnet50_cifar100_int8_final_32x32_200.pth')
    # TODO ONNX integration
if __name__ == '__main__':
    main()

Files already downloaded and verified
Files already downloaded and verified


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
Calibrating:   0%|          | 0/352 [00:00<?, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/ao/quantization/fake_

Epoch 1/200 – Loss: 2.2384, ValAcc: 15.56%
Saved best QAT+EntKD model with ValAcc: 15.56%


Epoch 2/200 – Loss: 1.9454, ValAcc: 25.18%
Saved best QAT+EntKD model with ValAcc: 25.18%


Epoch 3/200 – Loss: 1.7348, ValAcc: 29.84%
Saved best QAT+EntKD model with ValAcc: 29.84%


...

Epoch 198/200 – Loss: 0.2660, ValAcc: 75.00%
No improvement in validation accuracy. Patience counter: 9/20


Epoch 199/200 – Loss: 0.2660, ValAcc: 75.08%
No improvement in validation accuracy. Patience counter: 10/20


Epoch 200/200 – Loss: 0.2661, ValAcc: 74.76%
No improvement in validation accuracy. Patience counter: 11/20



Best Fake-Quantized Model – Test Accuracy: 74.73%
Average Fake-Quantized Inference Latency (GPU): 25.67 ms/batch


Best INT8 Model – Test Accuracy: 74.78%
Average INT8 Inference Latency (CPU): 130.35 ms/batch
